## user table에 password 넣기

In [57]:
import json
import pymysql

with open ('mysql.json', 'r') as file :
    config_str = file.read()
    
config = json.loads(config_str)

conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

- ENCRYPT PASSWORD

In [58]:
import hashlib
import base64

In [4]:
pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

- ENCODING with Base64

In [6]:
base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [7]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [8]:
len(hashed_pwd)

44

In [15]:
# password를 생성하는 함수
def gen_pwd(pwd) :
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hashed_pwd

In [21]:
# users table을 읽어서 pwd열에 password 채우기
sql = """
    select uid from users;
"""
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',),
 ('djy',),
 ('eskim',),
 ('gdhong',),
 ('jbpark',),
 ('vincenzo',),
 ('wjlee',))

In [22]:
sql = """
    update users set pwd = %s where uid = %s;
"""
cur = conn.cursor()
for i in results :
    uid = i[0]
    pwd = gen_pwd(uid + '12')
    cur.execute(sql, (pwd, uid))
conn.commit()

## LOGIN

CASE 1) UID PASS and PWD PASS

In [24]:
uid, pwd = 'admin', 'admin12'

sql = "select uid, pwd from users where uid = %s;"
cur = conn.cursor()
cur.execute(sql, (uid,))
result = cur.fetchone()
db_pwd = result[1]
db_pwd

'EUZjqxlO3LP2HUCYg85K5sPC+YVBlAlaU4UBHRW+y+8='

In [25]:
hashed_pwd = gen_pwd(pwd)
hashed_pwd

'EUZjqxlO3LP2HUCYg85K5sPC+YVBlAlaU4UBHRW+y+8='

CASE 2) UID PASS and PWD FAIL

In [30]:
pwd = '1234'
hashed_pwd = gen_pwd(pwd)
if hashed_pwd != db_pwd :
    print('password is wrong. try again')

password is wrong. try again


CASE 3) UID FAIL and PWD FAIL

In [41]:
uid = 'fool'
sql = "select uid, pwd from users where uid = %s;"
cur = conn.cursor()
cur.execute(sql, (uid,))

0

In [42]:
result = cur.fetchone()
result

- MAKE LOGIN PROGRAMM

In [62]:
uid, pwd = 'admin', 'admin12'

In [63]:
sql = "select uid, pwd from users where uid = %s and is_deleted = 0;"
cur = conn.cursor()
cur.execute(sql, (uid,))
result = cur.fetchone()
try :
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd :
        print(f'Welcome {uid}!')
    else :
        print('Wrong password. Try agian')
except :
    print('Wrong uid. Try again')

Welcome admin!


In [64]:
cur.close()
conn.close()